In [2]:
# This cell should be exactly the same between all models

import pickle as pk

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from common import *
from util import *

# Load everything
scaled_data = {}
with np.load(PREPROCESSED_DATASET_FILEPATH) as npz_loader:
    for key in npz_loader:
        scaled_data[key] = npz_loader[key]
scaler = pk.load(open(PREPROCESSING_SCALER_FILEPATH, "rb"))

# Input and output dims
input_shape = tuple(list(scaled_data['x_train'].shape)[1:])
output_shape = tuple(list(scaled_data['y_train'].shape)[1:])
input_dims = np.product(input_shape)
output_dims = np.product(output_shape)

In [3]:
print("tensorflow: {}".format(tf.__version__))
print("numpy: {}".format(np.__version__))

tensorflow: 2.8.0
numpy: 1.21.5


In [4]:
# CONSTANTS:
MODEL_NAME = "LinearRegression"

In [5]:
# Construct TF model here

# Linear Regression
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=output_dims,
                              activation='linear',
                              kernel_regularizer=tf.keras.regularizers.L1L2(l1=0.0,l2=1e-3)),
        tf.keras.layers.Reshape(output_shape)
    ]
)

Metal device set to: Apple M1 Max


2022-03-17 14:29:28.756475: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-17 14:29:28.756759: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# hyperparameters and callbacks

BATCH_SIZE = 64
MAX_EPOCHS = 100

def scheduler(epoch, lr):
    if epoch<50:
        return lr
    elif epoch%20==0:
        return lr/10
    else:
        return lr
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(loss='mean_absolute_error',
              optimizer=tf.keras.optimizers.Adam(1e-3))

In [7]:
# Compile and fit model here
history = model.fit(
    x=scaled_data['x_train'],
    y=scaled_data['y_train'],
    batch_size=BATCH_SIZE,
    epochs=MAX_EPOCHS,
    validation_data=(scaled_data['x_val'],
    scaled_data['y_val']),
    callbacks=[callback_lr,callback_es]
)

Epoch 1/100


2022-03-17 14:29:33.293370: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-17 14:29:33.446327: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


570/570 [==============================] - ETA: 0s - loss: 0.7004

2022-03-17 14:29:46.070420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


570/570 [==============================] - 13s 23ms/step - loss: 0.7004 - val_loss: 0.5089 - lr: 0.0010
Epoch 2/100
570/570 [==============================] - 14s 24ms/step - loss: 0.4698 - val_loss: 0.5167 - lr: 0.0010
Epoch 3/100
570/570 [==============================] - 14s 25ms/step - loss: 0.4703 - val_loss: 0.5644 - lr: 0.0010
Epoch 4/100
570/570 [==============================] - 14s 25ms/step - loss: 0.4705 - val_loss: 0.4866 - lr: 0.0010
Epoch 5/100
570/570 [==============================] - 14s 25ms/step - loss: 0.4710 - val_loss: 0.5279 - lr: 0.0010
Epoch 6/100
570/570 [==============================] - 15s 25ms/step - loss: 0.4723 - val_loss: 0.5167 - lr: 0.0010
Epoch 7/100
570/570 [==============================] - 15s 26ms/step - loss: 0.4708 - val_loss: 0.4842 - lr: 0.0010
Epoch 8/100
570/570 [==============================] - 15s 26ms/step - loss: 0.4688 - val_loss: 0.5150 - lr: 0.0010
Epoch 9/100
570/570 [==============================] - 15s 26ms/step - loss: 0.4704 

In [8]:
# Compute unnormalized prediction loss

preds = {}

for split in ['test','val']:
    preds[split] = model.predict(scaled_data['x_'+split])
    preds[split] = scaler.inverse_transform(preds[split])

2022-03-17 14:34:17.521868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [9]:
# Save model
model.save(model_name_to_model_filepath(MODEL_NAME))

# Save run info
run_info = {}
run_info["history"] = history.history
run_info["predictions"] = preds # idk whether this part makes sense for RNNs or not
pk.dump(run_info, open(model_name_to_run_info_filepath(MODEL_NAME), "wb"))